# PROBLEM 1: Topic Models
## Obtain Topic Models (K=10, 20, 50) for both datasets by running LDA and NMF methods; you can call libraries for both methods and dont have to use the ES index as source. For both LDA and NMF: print out for each topic the top 20 words (with probabilities)

The rest of of topic exercises and results are required only for the LDA topics:

- 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. 

Does this change if one increases the topics from 20 to 50?


### 1. Data Preprocessing using gensim

In [2]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-download-auto-examples-tutorials-run-lda-py

# imports

from sklearn.datasets import fetch_20newsgroups
from gensim import corpora, models, similarities, downloader

# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# truncate the dataset to 1000 documents for faster processing
# newsgroups.data = newsgroups.data[:1000]

print(type(newsgroups.data[0][:]))  # <class 'scipy.sparse.csr.csr_matrix'>
print(newsgroups.data[2][:])  # <class 'numpy.ndarray'>



<class 'str'>
well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk rather than the 120, but i don't really have
a feel for how much "better" the display is (yea, it looks great in the
store, but is that all "wow" or i

In [3]:
# gensim expects a list of lists of tokens, so we need to convert the sparse matrix to a list of lists
# %pip install nltk
# Tokenize the documents
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

for idx in range(len(newsgroups.data)):
    newsgroups.data[idx] = newsgroups.data[idx][:].lower()  # convert to lowercase
    newsgroups.data[idx] = tokenizer.tokenize(newsgroups.data[idx])  # split into words

print(newsgroups.data[2]) 
print(len(newsgroups.data[2])) # number of tokens in the document

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [4]:
# Remove numbers, but not words that contain numbers.
for ng in newsgroups.data:
    ng = [token for token in ng if not token.isdigit()]
    # Remove words that are only one character.
    ng = [token for token in ng if len(token) > 1]



print(newsgroups.data[2])  
print(len(newsgroups.data[2]))

['well', 'folks', 'my', 'mac', 'plus', 'finally', 'gave', 'up', 'the', 'ghost', 'this', 'weekend', 'after', 'starting', 'life', 'as', 'a', '512k', 'way', 'back', 'in', '1985', 'sooo', 'i', 'm', 'in', 'the', 'market', 'for', 'a', 'new', 'machine', 'a', 'bit', 'sooner', 'than', 'i', 'intended', 'to', 'be', 'i', 'm', 'looking', 'into', 'picking', 'up', 'a', 'powerbook', '160', 'or', 'maybe', '180', 'and', 'have', 'a', 'bunch', 'of', 'questions', 'that', 'hopefully', 'somebody', 'can', 'answer', 'does', 'anybody', 'know', 'any', 'dirt', 'on', 'when', 'the', 'next', 'round', 'of', 'powerbook', 'introductions', 'are', 'expected', 'i', 'd', 'heard', 'the', '185c', 'was', 'supposed', 'to', 'make', 'an', 'appearence', 'this', 'summer', 'but', 'haven', 't', 'heard', 'anymore', 'on', 'it', 'and', 'since', 'i', 'don', 't', 'have', 'access', 'to', 'macleak', 'i', 'was', 'wondering', 'if', 'anybody', 'out', 'there', 'had', 'more', 'info', 'has', 'anybody', 'heard', 'rumors', 'about', 'price', 'drops

In [5]:
%pip install nltk
import nltk
# nltk.download()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\koola\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
# Lemmatize the documents
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
newsgroups.data = [[lemmatizer.lemmatize(token) for token in doc] for doc in newsgroups.data]

In [7]:
# Compute bigrams
from gensim.models import Phrases
bigrams = Phrases(newsgroups.data, min_count=5, threshold=100)
# bigrams = Phrases(newsgroups.data, min_count=20)
for idx in range(len(newsgroups.data)):
    for token in bigrams[newsgroups.data[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            newsgroups.data[idx].append(token)

In [8]:
# Removing words that appear in less than 20 docs OR in more than 50% of the docs.

# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(newsgroups.data)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
# dictionary.filter_extremes(no_below=20, no_above=0.5)

In [9]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in newsgroups.data]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 98832
Number of documents: 11314


### 2. Modeling

In [ ]:
# We are ready to train the LDA model. We will first discuss how to set some of the training parameters.

# chunksize 
    # controls how many documents are processed at a time in the training algorithm. 
    # Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. 
    # I’ve set chunksize = 2000, which is more than the amount of documents, so I process all the data in one go. 
    # Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.
# passes
    # controls how often we train the model on the entire corpus. 
    # Another word for passes might be “epochs”. iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document.
    # It is important to set the number of “passes” and “iterations” high enough.

from gensim.models import LdaModel

# Raghu will probably ask what params i am passing to the model
# Set training parameters.
num_topics = 20
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

# Train the model.
model = LdaModel(corpus=corpus, 
                 id2word=id2word, 
                 chunksize=chunksize, 
                 passes=passes, 
                 iterations=iterations, 
                 num_topics=num_topics, 
                 eval_every=eval_every) # for different num_topics? (solution to 50?)


In [11]:
# Print the Keyword in the topics -> K = 10
topics = model.print_topics(num_topics=10 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(6, '0.003*"gregory" + 0.003*"eugene" + 0.002*"alert" + 0.002*"netherlands" + 0.002*"471" + 0.001*"ist" + 0.001*"gameboy" + 0.001*"uta_edu" + 0.001*"uta" + 0.001*"matloff" + 0.001*"skiing" + 0.000*"starflight" + 0.000*"discreet" + 0.000*"cs1442au" + 0.000*"decster" + 0.000*"rao" + 0.000*"usma" + 0.000*"melbourne" + 0.000*"usma1" + 0.000*"usma1_usma"')
(4, '0.014*"navy" + 0.012*"plot" + 0.010*"symbol" + 0.008*"iraqi" + 0.008*"disaster" + 0.007*"syrian" + 0.005*"india" + 0.005*"plotting" + 0.005*"islander" + 0.004*"toll" + 0.003*"navy_mil" + 0.002*"moron" + 0.002*"kuwait" + 0.002*"1280x1024" + 0.002*"mil" + 0.002*"t560i" + 0.002*"sept" + 0.002*"viewsonic" + 0.002*"cambodia" + 0.001*"kuwaiti"')
(9, '0.048*"the" + 0.046*"of" + 0.039*"and" + 0.038*"in" + 0.015*"were" + 0.012*"armenian" + 0.011*"wa" + 0.010*"by" + 0.010*"year" + 0.008*"from" + 0.008*"1993" + 0.008*"child" + 0.007*"april" + 0.007*"their" + 0.007*"at" + 0.006*"for" + 0.006*"university" + 0.006*"health" + 0.006*"city" + 0.006*"

In [12]:
# -> K = 20
topics = model.print_topics(num_topics=20 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.039*"the" + 0.033*"a" + 0.030*"i" + 0.024*"to" + 0.021*"and" + 0.020*"for" + 0.016*"is" + 0.015*"it" + 0.012*"with" + 0.012*"you" + 0.011*"on" + 0.011*"in" + 0.009*"have" + 0.009*"or" + 0.009*"this" + 0.009*"can" + 0.008*"of" + 0.008*"if" + 0.007*"drive" + 0.006*"window"')
(1, '0.039*"edu" + 0.024*"com" + 0.014*"list" + 0.014*"file" + 0.013*"available" + 0.013*"from" + 0.013*"ftp" + 0.012*"mail" + 0.012*"information" + 0.009*"send" + 0.009*"c" + 0.009*"pub" + 0.009*"request" + 0.008*"address" + 0.008*"faq" + 0.008*"server" + 0.008*"internet" + 0.008*"site" + 0.007*"email" + 0.007*"message"')
(2, '0.018*"brave" + 0.011*"lost" + 0.010*"kent" + 0.010*"won" + 0.007*"cheer" + 0.007*"macro" + 0.007*"cheer_kent" + 0.003*"malcolm" + 0.002*"burton" + 0.001*"weitek" + 0.001*"symantec" + 0.001*"spontaneous" + 0.001*"smoltz" + 0.000*"avery" + 0.000*"origins_" + 0.000*"03_03" + 0.000*"04_02" + 0.000*"03_02" + 0.000*"01_03" + 0.000*"03_01"')
(3, '0.048*"m" + 0.036*"_" + 0.029*"w" + 0.023*"1" 

In [13]:
# -> K = 50
topics = model.print_topics(num_topics=50 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.039*"the" + 0.033*"a" + 0.030*"i" + 0.024*"to" + 0.021*"and" + 0.020*"for" + 0.016*"is" + 0.015*"it" + 0.012*"with" + 0.012*"you" + 0.011*"on" + 0.011*"in" + 0.009*"have" + 0.009*"or" + 0.009*"this" + 0.009*"can" + 0.008*"of" + 0.008*"if" + 0.007*"drive" + 0.006*"window"')
(1, '0.039*"edu" + 0.024*"com" + 0.014*"list" + 0.014*"file" + 0.013*"available" + 0.013*"from" + 0.013*"ftp" + 0.012*"mail" + 0.012*"information" + 0.009*"send" + 0.009*"c" + 0.009*"pub" + 0.009*"request" + 0.008*"address" + 0.008*"faq" + 0.008*"server" + 0.008*"internet" + 0.008*"site" + 0.007*"email" + 0.007*"message"')
(2, '0.018*"brave" + 0.011*"lost" + 0.010*"kent" + 0.010*"won" + 0.007*"cheer" + 0.007*"macro" + 0.007*"cheer_kent" + 0.003*"malcolm" + 0.002*"burton" + 0.001*"weitek" + 0.001*"symantec" + 0.001*"spontaneous" + 0.001*"smoltz" + 0.000*"avery" + 0.000*"origins_" + 0.000*"03_03" + 0.000*"04_02" + 0.000*"03_02" + 0.000*"01_03" + 0.000*"03_01"')
(3, '0.048*"m" + 0.036*"_" + 0.029*"w" + 0.023*"1" 

In [14]:
# - 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. Does this change if one increases the topics from 20 to 50?
i = 0
labels = set(newsgroups.target)
# print(newsgroups.target_names)
for label in newsgroups.target_names:
    print(i, ": ", newsgroups.target_names[i])
    i += 1
# these do not correlate, sort them or use a dictionary to align them correctly. 

0 :  alt.atheism
1 :  comp.graphics
2 :  comp.os.ms-windows.misc
3 :  comp.sys.ibm.pc.hardware
4 :  comp.sys.mac.hardware
5 :  comp.windows.x
6 :  misc.forsale
7 :  rec.autos
8 :  rec.motorcycles
9 :  rec.sport.baseball
10 :  rec.sport.hockey
11 :  sci.crypt
12 :  sci.electronics
13 :  sci.med
14 :  sci.space
15 :  soc.religion.christian
16 :  talk.politics.guns
17 :  talk.politics.mideast
18 :  talk.politics.misc
19 :  talk.religion.misc


In [15]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -6.6515.
[([(0.07914201, 'the'),
   (0.043036897, 'of'),
   (0.035029616, 'a'),
   (0.033568904, 'to'),
   (0.025729131, 'and'),
   (0.023519708, 'is'),
   (0.0211461, 'in'),
   (0.01247023, 'that'),
   (0.012252181, 'for'),
   (0.011108573, 'be'),
   (0.010859357, 'are'),
   (0.008846691, 'this'),
   (0.008677347, 'it'),
   (0.007707614, 'by'),
   (0.00758602, 'or'),
   (0.0074104792, 'not'),
   (0.0068915007, 'on'),
   (0.0063942065, 'with'),
   (0.0057826135, 's'),
   (0.0053862724, 'an')],
  -0.599649266837716),
 ([(0.04758928, 'the'),
   (0.032316152, 'i'),
   (0.02914017, 'to'),
   (0.027270427, 'a'),
   (0.0233965, 'that'),
   (0.020412356, 'and'),
   (0.019215282, 'it'),
   (0.01781288, 'you'),
   (0.015861338, 'of'),
   (0.014366238, 'in'),
   (0.012601636, 'is'),
   (0.012287056, 't'),
   (0.009851212, 'wa'),
   (0.009749076, 'they'),
   (0.009700128, 'have'),
   (0.009339686, 's'),
   (0.008723925, 'not'),
   (0.008543691, 'but'),
   (0.008110557, 'h

In [16]:
# NMF: Idea of the algorithm is as follows: (obtained via documentation at https://radimrehurek.com/gensim/models/nmf.html)

# Init W, A and B matrices

# Input corpus
# Split corpus into batches

# for v in batches: 
    # infer h:
        # do coordinate gradient descent step to find h that minimizes (v - Wh) l2 norm
        # bound h so that it is non-negative

    # update A and B:
        # A = h.dot(ht)
        # B = v.dot(ht)

    # update W:
        # do gradient descent step to find W that minimizes 0.5*trace(W^t W A) - trace (W^t B) l2 norm

In [20]:
from gensim.models import Nmf
from gensim.corpora.dictionary import Dictionary

# train the model on the corpus from above:
nmf = Nmf(corpus=corpus, 
           num_topics=num_topics)

# # train the model on the corpus from above:
# nmf = Nmf(corpus=corpus, 
#            num_topics=num_topics, # K = 10,20,50 
#            id2word=id2word, 
#            chunksize=chunksize, 
#            passes=passes, 
#            iterations=iterations)

# print the topics
topics = nmf.print_topics(num_topics=10 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability


(3, '0.025*"58" + 0.017*"74" + 0.014*"43" + 0.013*"318" + 0.012*"30" + 0.011*"62" + 0.009*"63" + 0.007*"61" + 0.007*"260" + 0.006*"28" + 0.005*"642" + 0.005*"188" + 0.004*"151" + 0.004*"59" + 0.004*"355" + 0.003*"2603" + 0.003*"40" + 0.003*"127" + 0.003*"1029" + 0.003*"344"')
(9, '0.078*"394" + 0.024*"69" + 0.017*"0" + 0.010*"70" + 0.009*"403" + 0.008*"410" + 0.007*"916" + 0.007*"726" + 0.006*"2596" + 0.006*"725" + 0.006*"412" + 0.006*"58" + 0.006*"398" + 0.006*"1137" + 0.005*"912" + 0.005*"397" + 0.005*"4645" + 0.005*"401" + 0.005*"1100" + 0.004*"68"')
(10, '0.016*"74" + 0.013*"58" + 0.013*"170" + 0.012*"93" + 0.009*"30" + 0.008*"32" + 0.008*"44" + 0.008*"3" + 0.006*"43" + 0.006*"419" + 0.005*"297" + 0.005*"22" + 0.005*"686" + 0.005*"436" + 0.004*"478" + 0.004*"316" + 0.004*"61" + 0.003*"956" + 0.003*"3254" + 0.003*"4742"')
(12, '0.042*"100" + 0.020*"1152" + 0.017*"1059" + 0.016*"3332" + 0.014*"3263" + 0.014*"4691" + 0.013*"749" + 0.013*"957" + 0.013*"7426" + 0.013*"2014" + 0.012*"69"

In [21]:
# -> K = 20
topics = nmf.print_topics(num_topics=20 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability
    

(0, '0.076*"1477" + 0.043*"100" + 0.023*"69" + 0.022*"1470" + 0.021*"1152" + 0.019*"3263" + 0.019*"119" + 0.019*"899" + 0.013*"2014" + 0.012*"412" + 0.011*"754" + 0.011*"836" + 0.011*"2596" + 0.010*"7426" + 0.010*"749" + 0.010*"1368" + 0.009*"243" + 0.009*"0" + 0.009*"916" + 0.009*"3332"')
(1, '0.097*"6643" + 0.046*"7426" + 0.039*"403" + 0.027*"3332" + 0.025*"1059" + 0.023*"1368" + 0.023*"2596" + 0.021*"6658" + 0.019*"6603" + 0.017*"5517" + 0.014*"1543" + 0.013*"6913" + 0.012*"5977" + 0.012*"5139" + 0.009*"6015" + 0.009*"100" + 0.008*"0" + 0.007*"6606" + 0.007*"6661" + 0.007*"5995"')
(2, '0.060*"58" + 0.032*"61" + 0.025*"43" + 0.025*"74" + 0.023*"260" + 0.015*"30" + 0.014*"33" + 0.013*"318" + 0.011*"642" + 0.011*"718" + 0.011*"62" + 0.009*"32" + 0.009*"349" + 0.008*"3" + 0.008*"60" + 0.008*"68" + 0.008*"157" + 0.006*"644" + 0.006*"26" + 0.006*"7"')
(3, '0.025*"58" + 0.017*"74" + 0.014*"43" + 0.013*"318" + 0.012*"30" + 0.011*"62" + 0.009*"63" + 0.007*"61" + 0.007*"260" + 0.006*"28" + 0.

In [22]:
# -> K = 50
topics = nmf.print_topics(num_topics=50 ,num_words=20)
for topic in topics:
    print(topic)  # topic number, topic words, topic probability

(0, '0.076*"1477" + 0.043*"100" + 0.023*"69" + 0.022*"1470" + 0.021*"1152" + 0.019*"3263" + 0.019*"119" + 0.019*"899" + 0.013*"2014" + 0.012*"412" + 0.011*"754" + 0.011*"836" + 0.011*"2596" + 0.010*"7426" + 0.010*"749" + 0.010*"1368" + 0.009*"243" + 0.009*"0" + 0.009*"916" + 0.009*"3332"')
(1, '0.097*"6643" + 0.046*"7426" + 0.039*"403" + 0.027*"3332" + 0.025*"1059" + 0.023*"1368" + 0.023*"2596" + 0.021*"6658" + 0.019*"6603" + 0.017*"5517" + 0.014*"1543" + 0.013*"6913" + 0.012*"5977" + 0.012*"5139" + 0.009*"6015" + 0.009*"100" + 0.008*"0" + 0.007*"6606" + 0.007*"6661" + 0.007*"5995"')
(2, '0.060*"58" + 0.032*"61" + 0.025*"43" + 0.025*"74" + 0.023*"260" + 0.015*"30" + 0.014*"33" + 0.013*"318" + 0.011*"642" + 0.011*"718" + 0.011*"62" + 0.009*"32" + 0.009*"349" + 0.008*"3" + 0.008*"60" + 0.008*"68" + 0.008*"157" + 0.006*"644" + 0.006*"26" + 0.006*"7"')
(3, '0.025*"58" + 0.017*"74" + 0.014*"43" + 0.013*"318" + 0.012*"30" + 0.011*"62" + 0.009*"63" + 0.007*"61" + 0.007*"260" + 0.006*"28" + 0.